# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="tennis.app",no_graphics=True)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [5]:
'''
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))
'''

"\nfor i in range(1, 6):                                      # play game for 5 episodes\n    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    \n    states = env_info.vector_observations                  # get the current state (for each agent)\n    scores = np.zeros(num_agents)                          # initialize the score (for each agent)\n    while True:\n        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)\n        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1\n        env_info = env.step(actions)[brain_name]           # send all actions to tne environment\n        next_states = env_info.vector_observations         # get next state (for each agent)\n        rewards = env_info.rewards                         # get reward (for each agent)\n        dones = env_info.local_done                        # see if episode finished\n        scores += env_info.rewards               

When finished, you can close the environment.

### Training the network

In [6]:
import torch
import numpy as np
from collections import deque
import os
import matplotlib.pyplot as plt
%matplotlib inline

### Select multi agent or normal agent 

In [7]:
from maddpg_agent import Agent as MA
from ddpg_agent import Agent as DA
def select_agent(agent):
    if agent=='ddpg':
        return DA(state_size, action_size,num_agents,fc1=400,fc2=300, seed=0,update_times=10)
    elif agent=='maddpg':
        return MA(state_size, action_size,num_agents,fc1=400,fc2=300, seed=0,update_times=10)
    else:
        print("wrong selection. select from 1. ddpg, 2. maddpg")
agent=select_agent('maddpg')

### Solve environment

In [ ]:
scores = [] 

def solve_environment(n_episodes=6000):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
    """
                           # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    global scores
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name] # reset the environment
        agent.reset_random()              #reset noise object
        state = env_info.vector_observations
        
        score = 0
        t=0
        reward_this_episode_1=0
        reward_this_episode_2=0
        while True:
            t=t+1
            action=agent.act(state)
            env_info = env.step(np.array(action))[brain_name] 
            next_state = env_info.vector_observations   # get the next state
            reward = env_info.rewards                   # get the reward

            done = env_info.local_done
            #print(state[0])
            agent.step(state, action, reward, next_state, done)
            state = next_state
            #print(reward)
            reward_this_episode_1+=reward[0]
            reward_this_episode_2+=reward[1]
            
            if np.any(done):
                break 
 
        score = max(reward_this_episode_1,reward_this_episode_2)
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score

        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=2:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.critic_local.state_dict(), 'trained_weights/checkpoint_critic.pth')
            torch.save(agent.actor_local.state_dict(), 'trained_weights/checkpoint_actor.pth')
            break
    return 



solve_environment()

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

Episode 100	Average Score: 0.01
Episode 200	Average Score: 0.01
Episode 300	Average Score: 0.04
Episode 400	Average Score: 0.06
Episode 500	Average Score: 0.15
Episode 600	Average Score: 0.20
Episode 700	Average Score: 0.21
Episode 800	Average Score: 0.13
Episode 900	Average Score: 0.13
Episode 1000	Average Score: 0.09
Episode 1100	Average Score: 0.08
Episode 1200	Average Score: 0.13
Episode 1300	Average Score: 0.10
Episode 1400	Average Score: 0.15
Episode 1500	Average Score: 0.40
Episode 1600	Average Score: 0.24
Episode 1700	Average Score: 0.05
Episode 1800	Average Score: 0.08
Episode 1900	Average Score: 0.14
Episode 2000	Average Score: 0.09
Episode 2100	Average Score: 0.14
Episode 2200	Average Score: 0.11
Episode 2300	Average Score: 0.11
Episode 2400	Average Score: 0.13
Episode 2500	Average Score: 0.07
Episode 2600	Average Score: 0.06
Episode 2700	Average Score: 0.10
Episode 2800	Average Score: 0.15
Episode 2900	Average Score: 0.13
Episode 3000	Average Score: 0.15
Episode 3100	Averag

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()


### Load the trained actor and critic weights 

In [ ]:
agent.critic_local.load_state_dict(torch.load('trained_weights/checkpoint_critic.pth'))
agent.actor_local.load_state_dict(torch.load('trained_weights/checkpoint_actor.pth'))

### See your trained agent perform the task.

In [ ]:
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations            # get the current state
#print(state.shape)
score = 0    
reward_i=[]# initialize the score
past_a=deque(maxlen=5)
while True:
    action=[]

    action=agent.act(state)        # select an action
    env_info = env.step(np.array(action))[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations   # get the next state
    reward = env_info.rewards                  # get the reward
    done = env_info.local_done                 # see if episode has finished
    score += np.mean(reward)                   # update the score
    state = next_state                         # roll over the state to next time step

    if np.any(done):                           # exit loop if episode finished
        break
    
print("Score: {}".format(score))


In [ ]:
env.close()